In [1]:
import pandas as pd
import numpy
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from nltk.feature_extraction.text import TfidVectorizer

In [2]:
dataframe_train=pd.read_csv("nlp_train.csv")
dataframe_test=pd.read_csv("nlp_test.csv")
y_train=dataframe_train['airline_sentiment']
documents_train=dataframe_train['text']
documents_test=dataframe_test['text']

In [3]:
stop=stopwords.words('english')
stop=stop+list(string.punctuation)
stop.append("..")
stop.append("'")
stop.append("'s'")
def get_simple_pos(tag):
    if(tag.startswith('J')):
        return wordnet.ADJ
    elif(tag.startswith('V')):
        return wordnet.VERB
    elif(tag.startswith('N')):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN;

In [4]:
words_document_train=[word_tokenize(doc) for doc in documents_train]
words_document_test=[word_tokenize(doc) for doc in documents_test]

In [5]:
lemma=WordNetLemmatizer()
def clean_review(words):
    output_words=[]
    words=set(words)
    for w in words:
        if(w.lower() not in stop):
            pos1=pos_tag([w])
            clean_word=lemma.lemmatize(w,pos=get_simple_pos(pos1[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [7]:
clean_documents_train=[]
clean_documents_test=[]
for x in range(len(words_document_train)):
    clean_documents_train.append((clean_review(words_document_train[x]),y_train[x]))
for x in range(len(words_document_test)):
    clean_documents_test.append(clean_review(words_document_test[x]))    

In [8]:
clean_train=[" ".join(document) for document,category in clean_documents_train]
clean_test=[" ".join(document) for document in clean_documents_test]

In [9]:
count_vec=CountVectorizer(max_features=2000,stop_words=stop,ngram_range=(1,2))
a=count_vec.fit_transform(clean_train)
b=count_vec.transform(clean_test)
x_train=a.todense()
x_test=b.todense()

In [10]:
svm=SVC()
svm.fit(x_train,y_train)

SVC()

In [13]:
print(svm.score(x_train,y_train))

0.8982695810564663


In [14]:
y_pred=svm.predict(x_test)

In [16]:
pd.DataFrame(y_pred).to_csv("Prediction.csv",header=False,index=False)